In [1]:
"""
Test Naive RAG Implementation
This notebook tests the complete RAG pipeline step by step
"""

import sys
sys.path.insert(0, '../src')

from naive_rag import NaiveRAG
import time

print("Testing Naive RAG Implementation\n" + "="*50)

# Initialize
rag = NaiveRAG()
print("RAG system initialized")


import os
os.environ['HF_HOME'] = 'Z:/hf_cache'
os.environ['TRANSFORMERS_CACHE'] = 'Z:/hf_cache'
os.environ['HF_DATASETS_CACHE'] = 'Z:/hf_cache'
print("Cache redirected to Z: drive")

2025-10-02 00:12:56,924 - naive_rag - INFO - Initializing RAG with embedding model: all-MiniLM-L6-v2
2025-10-02 00:12:56,934 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2025-10-02 00:12:56,934 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Utils module loaded successfully!
Testing Naive RAG Implementation


2025-10-02 00:12:58,524 - naive_rag - INFO - RAG system initialized successfully


RAG system initialized
Cache redirected to Z: drive


In [2]:
# Load dataset
print("\n1. Loading dataset...")
start_time = time.time()

documents, qa_pairs = rag.load_dataset()

print(f"Dataset loaded in {time.time() - start_time:.2f} seconds")
print(f"   Documents: {len(documents)}")
print(f"   Q&A pairs: {len(qa_pairs)}")

2025-10-02 00:12:58,547 - naive_rag - INFO - Loading RAG Mini Wikipedia dataset...
2025-10-02 00:12:58,548 - naive_rag - INFO - Loading text corpus...



1. Loading dataset...


2025-10-02 00:13:02,061 - naive_rag - INFO - Loading Q&A pairs...
2025-10-02 00:13:03,800 - naive_rag - INFO - Loaded 3200 documents and 918 Q&A pairs


Dataset loaded in 5.25 seconds
   Documents: 3200
   Q&A pairs: 918


In [3]:
# Create embeddings
print("\n2. Creating embeddings...")
start_time = time.time()

embeddings = rag.create_embeddings(batch_size=32)

print(f"Embeddings created in {time.time() - start_time:.2f} seconds")
print(f"   Shape: {embeddings.shape}")
print(f"   Memory: ~{embeddings.nbytes / 1024 / 1024:.1f} MB")

2025-10-02 00:13:03,814 - naive_rag - INFO - Creating embeddings for 3200 documents...



2. Creating embeddings...


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

2025-10-02 00:14:58,037 - naive_rag - INFO - Created embeddings with shape: (3200, 384)


Embeddings created in 114.22 seconds
   Shape: (3200, 384)
   Memory: ~4.7 MB


In [4]:
# Build FAISS index
print("\n3. Building vector database...")
start_time = time.time()

vector_db = rag.build_vector_db()

print(f"Vector DB built in {time.time() - start_time:.2f} seconds")
print(f"   Total vectors: {vector_db.ntotal}")

2025-10-02 00:14:58,048 - naive_rag - INFO - Building FAISS vector database...
2025-10-02 00:14:58,054 - naive_rag - INFO - FAISS index built with 3200 vectors



3. Building vector database...
Vector DB built in 0.01 seconds
   Total vectors: 3200


In [20]:
#Testing retrieval
print("\n4. Testing retrieval...")

test_question = "Was Abraham Lincoln the sixteenth President of the United States?"
print(f"Question: {test_question}")

results = rag.retrieve(test_question, top_k=3)

print(f"\nRetrieved {len(results)} documents:")
for i, (doc, score) in enumerate(results, 1):
    print(f"\n--- Result {i} (score: {score:.4f}) ---")
    print(doc[:300] + "...")


4. Testing retrieval...
Question: Was Abraham Lincoln the sixteenth President of the United States?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Retrieved 3 documents:

--- Result 1 (score: 0.7095) ---
Young Abraham Lincoln...

--- Result 2 (score: 0.5840) ---
Abraham Lincoln (February 12, 1809 â April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination. As an outspoken opponent of the expansion of slavery in the United States, "[I]n his short autobiography written for the 1860 presidential camp...

--- Result 3 (score: 0.5569) ---
Sixteen months before his death, his son, John Quincy Adams, became the sixth President of the United States (1825 1829), the only son of a former President to hold the office until George W. Bush in 2001....


In [6]:
# Load generation model
print("\n5. Loading generation model...")
start_time = time.time()

rag.load_generator("google/flan-t5-base")

print(f"Generator loaded in {time.time() - start_time:.2f} seconds")

2025-10-02 00:14:58,131 - naive_rag - INFO - Loading generation model: google/flan-t5-base



5. Loading generation model...


2025-10-02 00:14:58,911 - naive_rag - INFO - Generation model loaded successfully


Generator loaded in 0.78 seconds


In [18]:
# Test answer generation
print("\n6. Testing answer generation...")

test_question = "Was Abraham Lincoln the sixteenth President of the United States?"
context = results[0][0]  # Use top retrieved document

answer = rag.generate_answer(test_question, context, prompt_strategy="basic")

print(f"Question: {test_question}")
print(f"Generated Answer: {answer}")
print(f"Expected Answer: yes")


6. Testing answer generation...
Question: Was Abraham Lincoln the sixteenth President of the United States?
Generated Answer: no
Expected Answer: yes


In [24]:
print("\n7. Testing complete RAG pipeline...")

test_questions = [
    "Was Abraham Lincoln the sixteenth President of the United States?",
    "When did Lincoln begin his political career?",
    "How long was Lincoln's formal education?"
]

for question in test_questions:
    result = rag.query(question, top_k=1, prompt_strategy="basic")
    print(f"\nQ: {result['question']}")
    print(f"A: {result['answer']}")
    print(f"Retrieval score: {result['retrieval_score']:.4f}")


7. Testing complete RAG pipeline...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Q: Was Abraham Lincoln the sixteenth President of the United States?
A: no
Retrieval score: 0.7095


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Q: When did Lincoln begin his political career?
A: 1832
Retrieval score: 0.6693


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Q: How long was Lincoln's formal education?
A: 18 months
Retrieval score: 0.6694
